In [7]:
import os
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [2]:
import numpy as np
from tqdm import tqdm
from dataset.patch_dataset import BrainPatchesDataModule
from models.UNetModule import UNet3
from dataset.roi_extraction import slice_image, reconstruct_patches
from utils import z_score_norm
import SimpleITK as sitk
import torch
from models.EM import ExpectationMaximization
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from brain_segmenter import BrainSegmenter

/home/user0/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Select GPU index to run the prediction on and path for the checkpoint used to load the model

In [3]:
DEVICE = 'cuda:2'
CHKP_PATH = Path('/home/user0/misa_vlex/brain_segmentation/outputs/unet3p_noaugm_bce_64_05/version_0/checkpoints/epoch=42-valid_dsc=0.0000.ckpt')

bsegm = BrainSegmenter(CHKP_PATH, DEVICE)

Model loaded


# Get Validation Results

In [4]:
val_path = Path('/home/user0/misa_vlex/brain_segmentation/data/Validation_Set')
val_path_res = val_path/f'unet_results/{bsegm.cfg["exp_name"]}/'
val_path_res.mkdir(exist_ok=True, parents=True)

results = []
for case in tqdm(val_path.iterdir(), total=5):
    if case.is_dir() and 'unet_results' not in case.name:
        
        # read images
        img = sitk.ReadImage(str(case/f'{case.name}.nii.gz'))
        imgnp = sitk.GetArrayFromImage(img)
        gt_seg = sitk.ReadImage(str(case/f'{case.name}_seg.nii.gz'))
        gt_segnp = sitk.GetArrayFromImage(gt_seg)
        
        # make prediction
        pred_seg, seg_res = bsegm.segment_and_compare(imgnp, gt_segnp)
        seg_res['case'] = case.name
        seg_res['model'] = bsegm.cfg["exp_name"]
        results.append(seg_res)
        
        # save prediction
        pred_seg_itk = sitk.GetImageFromArray(pred_seg)
        pred_seg_itk.CopyInformation(gt_seg)
        pred_seg_itk.SetOrigin(gt_seg.GetOrigin())
        pred_seg_itk.SetDirection(gt_seg.GetDirection())
        pred_seg_itk.SetSpacing(gt_seg.GetSpacing())
        sitk.WriteImage(pred_seg_itk, str(val_path_res/f'{case.name}_seg.nii.gz'))
        
results_df = pd.DataFrame(results)
results_df.to_csv(val_path_res/'results.csv', index=False)

0it [00:00, ?it/s]/home/user0/misa_vlex/brain_segmentation/utils.py:7: RuntimeWarning: Mean of empty slice.
  mean = img[img != 0].mean()
/home/user0/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/user0/.local/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/user0/.local/lib/python3.10/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/user0/.local/lib/python3.10/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/user0/misa_vlex/brain_segmentation/utils.py:15: RuntimeWarning: divide by zero encountered in divide
  img = (img - mean) / std
/home/user0/misa

# Make Test Predictions

Be sure to have correct device and checkpoint path set up above

In [15]:
test_path = Path('/home/user0/misa_vlex/brain_segmentation/data/Test_Set')
test_path_res = test_path/f'unet_results/{bsegm.cfg["exp_name"]}/'
test_path_res.mkdir(exist_ok=True, parents=True)

results = []
for case in tqdm(test_path.iterdir(), total=3):
    if case.is_dir() and 'unet_results' not in case.name:
        
        # read images
        img = sitk.ReadImage(str(case/f'{case.name}.nii.gz'))
        imgnp = sitk.GetArrayFromImage(img)
        
        # make prediction
        pred_seg = bsegm.segment(imgnp, progress=False)

        # save prediction
        pred_seg_itk = sitk.GetImageFromArray(pred_seg)
        pred_seg_itk.CopyInformation(img)
        pred_seg_itk.SetOrigin(img.GetOrigin())
        pred_seg_itk.SetDirection(img.GetDirection())
        pred_seg_itk.SetSpacing(img.GetSpacing())
        sitk.WriteImage(pred_seg_itk, str(test_path_res/f'{case.name}_seg.nii.gz'))

  0%|          | 0/3 [00:42<?, ?it/s]
